In [ ]:
from pathlib import Path

from prettytable import PrettyTable
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from libratom.lib.database import db_session
from libratom.models import HeaderField, HeaderFieldType, Message

In [ ]:
db_file_name = "report.sqlite3"

In [ ]:
db_file = Path(db_file_name).resolve()
db_file.unlink(missing_ok=True)

### Run command line tool

In [ ]:
!ratom report -mvp -o {db_file_name} data/RevisedEDRMv1_Complete/albert_meyers/

### Examine DB output

#### Using the ORM

In [ ]:
engine = create_engine(f"sqlite:///{db_file}")
session = sessionmaker(bind=engine)()

See also: [db_session_from_cmd_out()](https://github.com/libratom/libratom/blob/0.6.0/libratom/lib/database.py#L55)

In [ ]:
# Quick check on the total number of messages
session.query(Message).count()

In [ ]:
# From here on we'll load DB table contents in memory
# and work with them in Python for convenience
messages, header_fields, header_field_types = session.query(Message).all(), session.query(HeaderField).all(), session.query(HeaderFieldType).all()

In [ ]:
# Total number of header fields
len(header_fields)

In [ ]:
# Header field count by type
for type_ in header_field_types:
    if count := len([field for field in header_fields if field.header_field_type == type_]):
        print(f"{type_.name}: {count}")

In [ ]:
# Pick a given message
message = session.query(Message).first()
message.pff_identifier

In [ ]:
# Raw header block from PST archive, for that message
print(message.headers)

In [ ]:
# Header fields for that message
results = PrettyTable(field_names=["name", "value"], align="l")

for header in message.header_fields:
    results.add_row([header.name, header.value])

print(results)

In [ ]:
# Any message with no header fields?
[message.pff_identifier for message in messages if not message.header_fields]

In [ ]:
session.close()

#### Using direct SQL queries

In [ ]:
# Message count
!sqlite3 -box {db_file} "select count(*) as message_count from message;"

In [ ]:
# Header field count
!sqlite3 -box {db_file} "select count(*) as header_count from header_field;"

In [ ]:
# Header field count by type
!sqlite3 -box {db_file} "\
select count(*) as header_count, name \
from header_field \
inner join header_field_type \
on header_field_type.id = header_field.header_field_type_id \
group by header_field_type_id; \
"

In [ ]:
# Pick a given message and confirm it's the same one as above
!sqlite3 -box {db_file} "select id, pff_identifier from message limit 1;"

In [ ]:
# Raw header block for that message
!sqlite3 {db_file} "select headers from message where id=1;"

In [ ]:
# Header fields for that message
!sqlite3 -box {db_file} "\
select name, value \
from header_field \
inner join header_field_type \
on header_field.header_field_type_id = header_field_type.id \
inner join message \
on message.id = header_field.message_id \
where message.id = 1; \
"